In [25]:
import pandas as pd
import numpy as np

#NLTK
import nltk
from nltk import word_tokenize, WordPunctTokenizer, regexp_tokenize
from nltk import word_tokenize, WordPunctTokenizer, regexp_tokenize

#Plotting 
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

#Keras
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

# Data de entrenamiento

In [26]:
import xml.etree.ElementTree as ET
tree = ET.parse('train2.xml')
root = tree.getroot()

In [27]:
train_set = pd.DataFrame({'tweet_id':[],'tweetText':[],'polarity_value':[],'polarity_type':[],'topic':[]})
row=0
for tweet in root:
    tweet_id = 'ID:'+tweet.find('tweetid').text
    tweetText = tweet.find('content').text
    lang = tweet.find('lang').text
    polarity_value = tweet.find('sentiments').find('polarity').find('value').text
    polarity_type = tweet.find('sentiments').find('polarity').find('type').text
    topic = tweet.find('topics').find('topic').text
    
    if lang == 'es':
        train_set.loc[row] = [tweet_id,tweetText,polarity_value,polarity_type,topic]
        row+=1

In [28]:
train_set['set']='train'

In [29]:
#Mostrando la data
train_set.head(10)

,tweet_id,tweetText,polarity_value,polarity_type,topic,set
0,ID:142389495503925248,"Salgo de #VeoTV , que día más largoooooo...",NONE,AGREEMENT,otros,train
1,ID:142389933619945473,@PauladeLasHeras No te libraras de ayudar me/n...,NEU,DISAGREEMENT,otros,train
2,ID:142391947707940864,@marodriguezb Gracias MAR,P,AGREEMENT,otros,train
3,ID:142416095012339712,"Off pensando en el regalito Sinde, la que se v...",N+,AGREEMENT,política,train
4,ID:142422495721562112,Conozco a alguien q es adicto al drama! Ja ja ...,P+,AGREEMENT,otros,train
5,ID:142424715175280640,"RT @FabHddzC: Si amas a alguien, déjalo libre....",NONE,AGREEMENT,música,train
6,ID:142483342040907776,Toca @crackoviadeTV3 . Grabación dl especial N...,P+,AGREEMENT,entretenimiento,train
7,ID:142493511634259968,Hoy asisitiré en Madrid a un seminario sobre l...,NONE,AGREEMENT,política,train
8,ID:142494476051562496,Buen día todos! Lo primero mandar un abrazo gr...,P+,AGREEMENT,otros,train
9,ID:142496796416016384,Desde el escaño. Todo listo para empezar #endi...,P+,AGREEMENT,política,train


In [30]:
train_set.columns = ['tweet_id', 'tweetText', 'polarity_value', 'polarity_type', 'topic','set']

In [31]:
train_set.head(4)

,tweet_id,tweetText,polarity_value,polarity_type,topic,set
0,ID:142389495503925248,"Salgo de #VeoTV , que día más largoooooo...",NONE,AGREEMENT,otros,train
1,ID:142389933619945473,@PauladeLasHeras No te libraras de ayudar me/n...,NEU,DISAGREEMENT,otros,train
2,ID:142391947707940864,@marodriguezb Gracias MAR,P,AGREEMENT,otros,train
3,ID:142416095012339712,"Off pensando en el regalito Sinde, la que se v...",N+,AGREEMENT,política,train


In [32]:
train_set.topic.value_counts()

política           2715
otros              1620
entretenimiento    1209
economía            525
música              412
fútbol              225
cine                183
tecnología          144
deportes            101
literatura           84
Name: topic, dtype: int64

In [33]:
train_set.polarity_value.value_counts()

P+      1652
NONE    1482
N       1335
P       1232
N+       847
NEU      670
Name: polarity_value, dtype: int64

In [34]:
train_set = train_set.loc[(train_set.polarity_value == 'P') | (train_set.polarity_value == 'N') ]

# Tweets Peruanos 

In [35]:
tweets=pd.read_json('file.json',lines=True)
tweets['set']='test'
tweets['polarity_value']=np.NaN

In [36]:
#Dimensiones
tweets.shape

(1026, 38)

# Procesando Tweets

In [37]:
processed_tweets= pd.concat([pd.DataFrame({'tweetID':tweets.id, 'tweetText':tweets.tweet,'polarity_value':tweets.polarity_value, 'set':tweets.set}), 
                             pd.DataFrame({'tweetID':train_set.tweet_id, 'tweetText':train_set.tweetText, 'polarity_value':train_set.polarity_value,'set':train_set.set})], ignore_index=True)
processed_tweets['processed_tweet'] = processed_tweets.tweetText
processed_tweets.sample(4)

,tweetID,tweetText,polarity_value,set,processed_tweet
1562,ID:153901981797335040,"Se congelan el sueldo de los funcionarios, per...",N,train,"Se congelan el sueldo de los funcionarios, per..."
510,1328024276679749633,SE LOGRO!!!!! RENUNCIO ESA RATA!!!! #MERINONOM...,NaN,test,SE LOGRO!!!!! RENUNCIO ESA RATA!!!! #MERINONOM...
1608,ID:154717637895655426,Urdangarin pagó a la pareja rumana q atendía e...,N,train,Urdangarin pagó a la pareja rumana q atendía e...
1958,ID:161957420409688064,"Levante y Mirandés, dos modestos rompiendo mol...",P,train,"Levante y Mirandés, dos modestos rompiendo mol..."


## Hashtags

In [38]:
import re
hash_regex = re.compile(r"#(\w+)")
hstgs = [] # To store the hashtags so we can exclude them from some parts of the analysis
def hash_repl(match):
    _ = '__HASH_'+match.group(1).upper()
    hstgs.append(_)
    return _

## USERNAMES

In [39]:
user_regex = re.compile(r"@(\w+)")
usr_names = [] # To store the user names so we can exclude them from some parts of the analysis
def user_repl(match):
    _ = '__user_'+match.group(1).upper()
    usr_names.append(_)
    return _

## URLs

In [40]:
url_regex = re.compile(r"(http|https|ftp)://[a-zA-Z0-9\./]+")
def url_repl(match):
    return '__URL_'


## REPETI

In [41]:
# Repeating words like hurrrryyyyyy
rpt_regex = re.compile(r"(.)\1{1,}", re.IGNORECASE);
def rpt_repl(match):
    return match.group(1)+match.group(1)

In [42]:
# Test
re.sub(rpt_regex, rpt_repl, "Reppppeated characters in wordsssssssss" )

'Reppeated characters in wordss'

## Puntuacion

In [43]:
# Spliting by word boundaries
word_bound_regex = re.compile(r"\W+")

# Punctuations
punctuations = \
	[	#('',		['.', ] )	,\
		#('',		[',', ] )	,\
		#('',		['\'', '\"', ] )	,\
		('__PUNC_EXCL',		['!', '¡', ] )	,\
		('__PUNC_QUES',		['?', '¿', ] )	,\
		('__PUNC_ELLP',		['...', '…', ] )	,\
	]

#For punctuation replacement
def punctuations_repl(match):
	text = match.group(0)
	repl = []
	for (key, parr) in punctuations :
		for punc in parr :
			if punc in text:
				repl.append(key)
	if( len(repl)>0 ) :
		return ' '+' '.join(repl)+' '
	else :
		return ' '

<function __main__.punctuations_repl(match)>

## Stemming

In [50]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')

In [51]:
# Test
text = processed_tweets.processed_tweet[55]
print(text)
text = [word if(word[0:2]=='__') else word.lower() for word in text.split() if ((len(word) >= 3) and (word not in usr_names))]
text = [stemmer.stem(w) for w in text]                
text = " ".join(text)
print('------------------')
print(text)

@MerinoDeLama Tienes el repudio de gran parte de la población peruana. #merinonomerepresenta
------------------
@merinodelam tien repudi gran part poblacion peruana. #merinonomerepresent


In [52]:
def sb_stem(text, only_first=0):
    text = [word if(word[0:2]=='__') else word.lower() for word in text.split() if ((len(word) >= 3) or (word in ['no','si', 'sí', 'ni']))] #   If we are doing negation analysis, maybe is a better idea to keep the small words (like 'no')
    text = [stemmer.stem(w) if w[0:2]!='__' else w for w in text ]
    
    return(text)

In [53]:

def processAll(text):
    text = re.sub( hash_regex, hash_repl, text )
    text = re.sub( user_regex, user_repl, text)
    text = re.sub( url_regex, url_repl, text )
    
    text = text.replace('\'','')
    
    text = re.sub( word_bound_regex , punctuations_repl, text )
    text = re.sub( rpt_regex, rpt_repl, text )
    
    text = sb_stem(text)    
    return text

In [55]:
d="Aasda!.? @Hola #AeaManito https://facebo.com asd"
processAll(d)

['aasd',
 '__PUNC_EXCL',
 '__PUNC_QUES',
 '__user_HOLA',
 '__HASH_AEAMANITO',
 '__URL_',
 'asd']

# Procesamiento de tweets

In [24]:
processed_tweets['processed_tweet'] = processed_tweets.tweetText.apply(processAll)


In [25]:
processed_tweets.sample(4)

,tweetID,tweetText,polarity_value,set,processed_tweet
3112,ID:179860484135206912,"El lunes 26, estreno de El Número Uno http://t...",P,train,"[lun, estren, numer, uno, __URL_, via, __user_..."
1266,ID:147716117274701825,Que buena noticia. ;) RT @FeerAlvizar: Primer ...,N,train,"[que, buen, notici, __user_FEERALVIZAR, prim, ..."
1103,ID:143987608870060032,BUENOS DÍAS PARTY TIME!!!,P,train,"[buen, dias, party, tim, __PUNC_EXCL]"
1403,ID:149832675094577152,"Cómpralo, no te arrepentirás, Nacho “@nachober...",P,train,"[compral, no, arrepent, nach, __user_NACHOBERJ..."


## Funciones

### Emoticones

In [26]:
# Emoticons
emoticons = \
	[	# For __EMOT_SMILEY
        (' __emoji: U+1F601',	[':-)', ':)', '(:', '(-:', ] )	,\
        # for __EMOT_LAUGH
		(' __emoji: U+1F923',		[':-D', ':D', 'X-D', 'XD', 'xD', ] )	,\
        # For __EMOT_LOVE
		(' __emoji: U+2764',		['<3', ':\*', ] )	,\
        # For __EMOT_WINK
		('__emoji: U+1F609',		[';-)', ';)', ';-D', ';D', '(;', '(-;', ] )	,\
        # For __EMOT_FROWN
		(' __emoji: U+2639',		[':-(', ':(', '(:', '(-:', ] )	,\
        # For __EMOT_CRY
		(' __emoji: U+1F622',		[':,(', ':\'(', ':"(', ':(('] )	,\
	]
    
def escape_paren(arr):
	return [text.replace(')', '[)}\]]').replace('(', '[({\[]') for text in arr]

def regex_union(arr):
	return '(' + '|'.join( arr ) + ')'

emoticons_regex = [ (repl, re.compile(regex_union(escape_paren(regx))) ) for (repl, regx) in emoticons ]

In [27]:
# Test
text = "This is a text with one emoticon :) and another :("
for (repl, regx) in emoticons_regex :
    text = re.sub(regx, ' '+repl+' ', text)
    
print(text)

This is a text with one emoticon   __emoji: U+1F601  and another   __emoji: U+2639 


In [28]:
emojis_db=pd.read_csv('emojis_db_csv.csv')
emojis_db.head()

,emoji_category,number,code,CLDR_Short_Name
0,face-positive,1,U+1F600,grinning face
1,face-positive,2,U+1F601,beaming face with smiling eyes
2,face-positive,3,U+1F602,face with tears of joy
3,face-positive,4,U+1F923,rolling on the floor laughing
4,face-positive,5,U+1F603,grinning face with big eyes


In [29]:
def emojis_unicode(tweet):
    ''' Extracts the emojis on the tweet on Unicode format, also tries to match those in regular format, such as ";)" '''
    for (repl, regx) in emoticons_regex :
        tweet = re.sub(regx, ' '+repl+' ', tweet)
#     print(tweet)

    tweet_unicode = str(tweet.encode('unicode-escape'))
    tweet_unicode = tweet_unicode.replace('\\\\U000',' __emoji: U+')
#     print(tweet_unicode)
    
    emoji_list = []
#     print(tweet)
    for emoji in range(tweet_unicode.count(' __emoji: ')):
        em = tweet_unicode.split('__emoji: ')[emoji+1].split()[0]
        em = em[:7] # the len of the emoji in unicode is between 6 and 7
        emoji_list.append(em.upper())
    return emoji_list

In [30]:
# Test
text = "😡🤣😂"
emojis_unicode(text)

['U+1F621', 'U+1F923', 'U+1F602']

In [31]:
def emoji_category(emojis):
    categories = []
    for i in range(len(emojis)):
        # print(emojis[i])
        try:
            categories.append(emojis_db.loc[emojis_db.code == emojis[i]].emoji_category.values[0])
        except:
            try:
                _ = emojis[i].split('+')[1] + '+'
                categories.append(emojis_db[emojis_db.code.str.contains(_)].emoji_category.values[0])
            except:
                categories.append('other')
    if len(categories) < 1:
        categories.append('no_emojis')
    return categories

emoji_category(emojis_unicode(text))

['face-negative', 'face-positive', 'face-positive']

# N- GRAMAS 

In [32]:

# Unigrams
unigrams_fd = nltk.FreqDist()
# unigrams_fd.update(text)
# unigrams_fd

# Bigrams
# words_bi  = [ ','.join(map(str,bg)) for bg in nltk.bigrams(text) ]
bi_grams_fd = nltk.FreqDist()
# bi_grams_fd.update( words_bi )
# bi_grams_fd

# Trigrams
# words_tri  = [ ','.join(map(str,tg)) for tg in nltk.trigrams(text) ]
tri_grams_fd = nltk.FreqDist()
# tri_grams_fd.update( words_tri )
# tri_grams_fd

In [33]:

# Wrapper function that encloses all the n-grams procedures
def get_word_features(words):
    bag = {}
    words_uni = [ 'has(%s)'% ug for ug in words ]
    words_bi  = [ 'has(%s)'% ','.join(map(str,bg)) for bg in nltk.bigrams(words) ]
    words_tri = [ 'has(%s)'% ','.join(map(str,tg)) for tg in nltk.trigrams(words) ]
    
    for f in words_uni+words_bi+words_tri:
        bag[f] = 1

    return bag

### Negacion

In [34]:
negtn_regex = re.compile( r"""(?:
    ^(?:nunca|no|nada|ningún|ninguno|ninguna|tampoco|
        nunc|nad|ningun|tampoc
    )$
)
|
n't
""", re.X)

def get_negation_features(words):
    INF = 0.0
    negtn = [ bool(negtn_regex.search(w)) for w in words ]

    left = [0.0] * len(words)
    prev = 0.0
    for i in range(0,len(words)):
        if( negtn[i] ):
            prev = 1.0
        left[i] = prev
        prev = max( 0.0, prev-0.1)

    right = [0.0] * len(words)
    prev = 0.0
    for i in reversed(range(0,len(words))):
        if( negtn[i] ):
            prev = 1.0
        right[i] = prev
        prev = max( 0.0, prev-0.1)

    return dict( zip(
                    ['neg_l('+w+')' for w in  words] + ['neg_r('+w+')' for w in  words],
                    left + right ) )

In [35]:
# Test
text = ["Este","tweet", "no", "es", "positivo"]
get_negation_features(text)


{'neg_l(Este)': 0.0,
 'neg_l(tweet)': 0.0,
 'neg_l(no)': 1.0,
 'neg_l(es)': 0.9,
 'neg_l(positivo)': 0.8,
 'neg_r(Este)': 0.8,
 'neg_r(tweet)': 0.9,
 'neg_r(no)': 1.0,
 'neg_r(es)': 0.0,
 'neg_r(positivo)': 0.0}

## Polarity

In [36]:
opener_lexicon = pd.read_csv('https://raw.githubusercontent.com/opener-project/public-sentiment-lexicons/master/propagation_lexicons/es/es.lemma.sy.an.hypo.rels.maxdepth5.seed500.maj.gold.csv', sep=';', header=None)
opener_lexicon.columns = ['x','pos', 'polarity', 'xx','word','xxx']
opener_lexicon.head()

,x,pos,polarity,xx,word,xxx
0,unknown,n,positive,1.0,felicitación,1
1,unknown,a,negative,1.0,inconsciente,1
2,unknown,n,neutral,1.0,soporte,1
3,unknown,n,neutral,1.0,proceso_matemático,1
4,unknown,v,positive,1.0,manifestar,1


In [37]:
def get_sent(polarity):
    if polarity == 'neutral':
        return 0
    if polarity == 'negative':
        return -1
    if polarity == 'positive':
        return 1
opener_lexicon['sentiment'] = opener_lexicon.polarity.apply(get_sent)

In [38]:
opener_lexicon['stmd_word'] = opener_lexicon['word'].apply(sb_stem)
opener_lexicon['stmd_word'] = opener_lexicon.stmd_word.apply(lambda x: str(x).replace('[','').replace(']','').replace("'",''))
opener_lexicon = opener_lexicon[['word','stmd_word','sentiment']]
opener_lexicon = opener_lexicon.loc[opener_lexicon.sentiment != 0]
opener_lexicon = opener_lexicon.drop_duplicates()
opener_lexicon.sample(4)

,word,stmd_word,sentiment
8143,cofirmar,cofirm,1.0
2804,investigador_privado,investigador_priv,1.0
4107,incrementar_la_intensidad,incrementar_la_intens,1.0
10594,defensividad,defens,1.0


In [39]:
def get_polarity_features(text):
    pol = {}
    text = sb_stem(text)
    for word in text:
        try:
            pol["sent("+word+")"] = opener_lexicon.loc[opener_lexicon.stmd_word == word].sentiment.values[0]
        except:
            continue
            
    return pol

_ = processed_tweets.tweetText[1503]
print('Tweet ',_)
print('Would have the following words and sentiments associated:')
print(get_polarity_features(_))

Tweet  ajá! “@JulenAriza: mal síntoma, especialmente para Newt Gingrich, que no ha podido conseguir 10.000 firmas en el mismo estado donde vive.”
Would have the following words and sentiments associated:
{'sent(mal)': -1.0, 'sent(especial)': 1.0, 'sent(pod)': -1.0, 'sent(consegu)': 1.0, 'sent(firm)': 1.0}


# Aplicando los features 

In [40]:
# Wrapper function for the extraction of features
def extract_features(text):
    global usr_names, hstgs
    features = {}
    words = text
    words = processAll(text)

    word_features = get_word_features(words)
    features.update( word_features )

    negation_features = get_negation_features(words)
    features.update( negation_features )
    
    # Sentiment features are not included on the final deliverabe as did not improve results
#     sentiment_features = get_polarity_features(text)
#     features.update(sentiment_features )
    
    emoji_features = emoji_category(emojis_unicode(text))
    emoji_features_dic = dict( zip(['emoji_('+w+')' for w in  emoji_features], emoji_features))
    features.update( emoji_features_dic )
    
    usr_names = list(set(usr_names))
    hstgs = list(set(hstgs))
    return features

In [41]:
processed_tweets['processed_tweet_features'] = processed_tweets.tweetText.apply(extract_features)

In [42]:
processed_tweets.processed_tweet_features[150]


{'has(dej)': 1,
 'has(que)': 1,
 'has(hic)': 1,
 'has(sea)': 1,
 'has(olvid)': 1,
 'has(__HASH_MERINONOMEREPRESENTA)': 1,
 'has(__HASH_NUEVACONSTITUCION)': 1,
 'has(__URL_)': 1,
 'has(dej,que)': 1,
 'has(que,que)': 1,
 'has(que,hic)': 1,
 'has(hic,sea)': 1,
 'has(sea,olvid)': 1,
 'has(olvid,__HASH_MERINONOMEREPRESENTA)': 1,
 'has(__HASH_MERINONOMEREPRESENTA,__HASH_NUEVACONSTITUCION)': 1,
 'has(__HASH_NUEVACONSTITUCION,__URL_)': 1,
 'has(dej,que,que)': 1,
 'has(que,que,hic)': 1,
 'has(que,hic,sea)': 1,
 'has(hic,sea,olvid)': 1,
 'has(sea,olvid,__HASH_MERINONOMEREPRESENTA)': 1,
 'has(olvid,__HASH_MERINONOMEREPRESENTA,__HASH_NUEVACONSTITUCION)': 1,
 'has(__HASH_MERINONOMEREPRESENTA,__HASH_NUEVACONSTITUCION,__URL_)': 1,
 'neg_l(dej)': 0.0,
 'neg_l(que)': 0.0,
 'neg_l(hic)': 0.0,
 'neg_l(sea)': 0.0,
 'neg_l(olvid)': 0.0,
 'neg_l(__HASH_MERINONOMEREPRESENTA)': 0.0,
 'neg_l(__HASH_NUEVACONSTITUCION)': 0.0,
 'neg_l(__URL_)': 0.0,
 'neg_r(dej)': 0.0,
 'neg_r(que)': 0.0,
 'neg_r(hic)': 0.0,
 'ne

# Parte 1 : Clasificacion - Analisis de Sentimientos 

In [43]:
train_ = processed_tweets.loc[processed_tweets.set == 'train']
test_ = processed_tweets.loc[processed_tweets.set == 'test']

In [44]:
test_.head()

,tweetID,tweetText,polarity_value,set,processed_tweet,processed_tweet_features
0,1341206151204106241,ME PREGUNTO SI LOS QUE ESCRIBEN #ContraTodoYCo...,NaN,test,"[pregunt, los, que, escrib, __HASH_CONTRATODOY...","{'has(pregunt)': 1, 'has(los)': 1, 'has(que)':..."
1,1341031364569812992,@Cycl0p77 @patriciagamarra Si llevan un cartel...,NaN,test,"[__user_CYCL0P77, __user_PATRICIAGAMARRA, llev...","{'has(__user_CYCL0P77)': 1, 'has(__user_PATRIC..."
2,1340782914250928135,@marcogiovannic #MerinoNoMeRepresenta,NaN,test,"[__user_MARCOGIOVANNIC, __HASH_MERINONOMEREPRE...","{'has(__user_MARCOGIOVANNIC)': 1, 'has(__HASH_..."
3,1340649911495372800,@otravezandres @SomosElComercio Están en la pl...,NaN,test,"[__user_OTRAVEZANDRES, __user_SOMOSELCOMERCIO,...","{'has(__user_OTRAVEZANDRES)': 1, 'has(__user_S..."
4,1340501298681147392,#nofilter #natural #lovechristmas #lovemyself ...,NaN,test,"[__HASH_NOFILTER, __HASH_NATURAL, __HASH_LOVEC...","{'has(__HASH_NOFILTER)': 1, 'has(__HASH_NATURA..."
